### chatcompletition, Prompt Template, LLM api call

In [1]:
import minsearch
import json
import os

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
        
Index = minsearch.Index(
    text_fields = ['question','section','text'],
    keyword_fields = ['course']
)  

Index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3, 'section' : 0.4}

    results = Index.search(
        query=query,
        filter_dict={'course':'mlops-zoomcamp'},
        boost_dict=boost,
        num_results=5
)
    return results

In [4]:
def build_prompt(query, search_results):
    prompt_template = """ 
You are an expert machine learning and mlops engineering helping a junior engineer as an assitant and guide. 
Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering. DO NOT USE OTHER CONTENT OTHER THAN GIVEN CONTEXT!
if the CONTEXT does not contain the answer, Output "Not FOUND in the context given" and explain your answer with reasons.

QUESTION: {question}

CONTEXT: {context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [5]:
from openai import OpenAI

In [6]:
def llm_call(prompt):
    client =  OpenAI(
        base_url = "http://localhost:11434/v1",
        api_key = "ollama"
)
    response = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt,}],
    model="phi3",
)
    return response.choices[0].message.content

In [7]:

def rag(query):
    
    search_results = search(query)
    Prompt = build_prompt(query, search_results)
    answer = llm_call(Prompt)
    return answer

In [12]:
rag("How to use Kafka and docker?")

'The steps provided to set environment variables for AWS credentials in a docker command are as follows: You should use the `-e` flag with `docker run`, specifying each variable separately after it, like this - assuming your image is named "homework-04":\n\n```\ndocker run -it --rm -v ~/.aws:/root/.aws homework-04 \\\n  -e AWS_ACCESS_KEY_ID=xxxxxxxx -e AWS_SECRET_ACCESS_KEY=xxxxx\n```  \nor by using the `--env` shorthand for `-e`. This tells Docker to pass these variables as environment variables into your container. Make sure that `homework-04` is replaced with your image name and you input real AWS access key, secret ID etc., when running this command on a secure system or after storing them safely in the `.aws/credentials` file.\nRemember not to store sensitive data like credentials directly into scripts for security purposes! It\'s also worth mentioning that some services might require additional setup (like `AWS_SESSION_TOKEN`) which would follow similar steps including passing as

In [13]:
print(_)

The steps provided to set environment variables for AWS credentials in a docker command are as follows: You should use the `-e` flag with `docker run`, specifying each variable separately after it, like this - assuming your image is named "homework-04":

```
docker run -it --rm -v ~/.aws:/root/.aws homework-04 \
  -e AWS_ACCESS_KEY_ID=xxxxxxxx -e AWS_SECRET_ACCESS_KEY=xxxxx
```  
or by using the `--env` shorthand for `-e`. This tells Docker to pass these variables as environment variables into your container. Make sure that `homework-04` is replaced with your image name and you input real AWS access key, secret ID etc., when running this command on a secure system or after storing them safely in the `.aws/credentials` file.
Remember not to store sensitive data like credentials directly into scripts for security purposes! It's also worth mentioning that some services might require additional setup (like `AWS_SESSION_TOKEN`) which would follow similar steps including passing as environme